In [2]:
import requests
import io
import pandas as pd
from PyPDF2 import PdfReader
from bs4 import BeautifulSoup


In [7]:
import requests
import io
import pandas as pd
from PyPDF2 import PdfReader
from bs4 import BeautifulSoup

def fetch_arxiv_data(search_query, max_results=10, start_date=None, end_date=None, primary_category=None, categories=None):
    # Define the API endpoint URL for searching articles
    api_url = "http://export.arxiv.org/api/query"

    # Define additional parameters like start and max_results
    params = {
        "search_query": search_query,
        "start": 0,  # Start index of results
        "max_results": max_results,  # Maximum number of results to retrieve
    }

    # Add start and end date parameters if provided
    if start_date is not None:
        params["start_date"] = start_date
    if end_date is not None:
        params["end_date"] = end_date

    # Add primary category and categories parameters if provided
    if primary_category is not None:
        params["cat"] = primary_category
    if categories is not None:
        params["categories"] = categories

    # Send an HTTP GET request to the arXiv API
    response = requests.get(api_url, params=params)

    # Check if the request was successful (HTTP status code 200)
    if response.status_code == 200:
        # Parse the response content using BeautifulSoup
        soup = BeautifulSoup(response.content, "xml")

        # Create a list to store article information
        articles = []

        # Extract article information from the parsed content
        entries = soup.find_all("entry")
        for entry in entries:
            title = entry.find("title").text
            paper_id = entry.find("id").text
            published = entry.find("published").text
            updated = entry.find("updated").text
            summary = entry.find("summary").text
            author = [author.text for author in entry.find_all("author")]
            comments = entry.find("arxiv:comment").text if entry.find("arxiv:comment") else ""
            journal_ref = entry.find("arxiv:journal_ref").text if entry.find("arxiv:journal_ref") else ""
            link = entry.find("link", title="pdf")["href"] if entry.find("link", title="pdf") else ""
            primary_category = entry.find("arxiv:primary_category")["term"] if entry.find("arxiv:primary_category") else ""
            categories = [cat["term"] for cat in entry.find_all("category")]
            doi = entry.find("arxiv:doi").text if entry.find("arxiv:doi") else ""
            license = entry.find("arxiv:license")["type"] if entry.find("arxiv:license") else ""
            affiliation = [aff.text for aff in entry.find_all("arxiv:affiliation")]

            # Append article information to the list
            articles.append({
                "Title": title,
                "ID": paper_id,
                "Published": published,
                "Updated": updated,
                "Summary": summary,
                "Author": author,
                "Comments": comments,
                "Journal_Ref": journal_ref,
                "Link": link,
                "Primary_Category": primary_category,
                "Categories": categories,
                "DOI": doi,
                "License": license,
                "Affiliation": affiliation,
            })

        # Create a DataFrame from the list of articles
        df = pd.DataFrame(articles)

        # Return the DataFrame
        return df

    else:
        print("Failed to retrieve data from arXiv API")
        return None

def extract_text_from_pdf(url):
    try:
        # Access the PDF from the URL
        response = requests.get(url)
        if response.status_code == 200:
            # Read the PDF content
            pdf_content = io.BytesIO(response.content)
            
            # Extract text from the PDF
            reader = PdfReader(pdf_content)
            text = ""
            for page in reader.pages:
                text += page.extract_text()
            
            return text
        else:
            print(f"Failed to retrieve content from {url}")
            return None
    except Exception as e:
        print(f"Error occurred while fetching content from {url}: {str(e)}")
        return None

# Fetch data from arXiv and store it in a DataFrame
df_arxiv = fetch_arxiv_data(search_query="quantum physics", max_results=10, start_date="2023-01-01", end_date="2023-12-31", primary_category="quant-ph", categories=["quant-ph", "cond-mat"])

# Generate new URLs with ".pdf" extension appended to the IDs
new_urls = [f"{id}.pdf" for id in df_arxiv['ID'].unique()]

# Extract text from the PDFs corresponding to the new URLs
pdf_contents = [extract_text_from_pdf(url) for url in new_urls]

# Create a new DataFrame with the URLs and extracted text
df_pdf_contents = pd.DataFrame({'URL': new_urls, 'Text': pdf_contents})

# Display the DataFrame
print(df_pdf_contents)

Error occurred while fetching content from http://arxiv.org/abs/quant-ph/0302169v1.pdf: EOF marker not found
Error occurred while fetching content from http://arxiv.org/abs/1212.4177v1.pdf: EOF marker not found
Error occurred while fetching content from http://arxiv.org/abs/1504.03207v1.pdf: EOF marker not found
Error occurred while fetching content from http://arxiv.org/abs/2208.08064v1.pdf: EOF marker not found
Error occurred while fetching content from http://arxiv.org/abs/cond-mat/0601285v1.pdf: EOF marker not found
Error occurred while fetching content from http://arxiv.org/abs/0811.2516v1.pdf: EOF marker not found
Error occurred while fetching content from http://arxiv.org/abs/1807.11019v1.pdf: EOF marker not found
Error occurred while fetching content from http://arxiv.org/abs/quant-ph/0201082v1.pdf: EOF marker not found
Error occurred while fetching content from http://arxiv.org/abs/quant-ph/0309066v1.pdf: EOF marker not found
Error occurred while fetching content from http://a